# E-commerce Data Analysis
---

**CASE**

An e-marketing entrepreneur uses specialized software to track customer ordering activity. 

Metrics tracked by the software include
* Historic number of orders
* Average order value
* Historic customer lifetime value
* Number of days active

This cient wants a value-add custom analysis of the data that goes beyond what the off-the-shelf software provides.

**OBJECTIVE**

Study the data in an intelligent way and provide fresh insights to inform client's forwad-looking decision-making. 

**APPROACH**

Here, I apply basic statistical analysis and compute Pearson's Correlation Coefficient (R) to aid client in evaluating the overall performance of client's marketing efforts.

**NOTE:** The dataset used in this demonstration has been altered from the original to preserve confidentiality.

In [15]:
import pandas as pd

from scipy.stats import pearsonr
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly.express as px
import plotly
from plotly.subplots import make_subplots

In [2]:
# pip install plotly==5.3.1

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Import the dataset as a Pandas dataframe
df = pd.read_csv('Demos/sales_data.csv)

In [6]:
# See what data types and how many records are present
df.shape[0]

2809

In [7]:
df.head()

,Historic Number Of Orders,Average Order Value,First Active,Last Active,Historic Customer Lifetime Value,days active
0,1.0,231.45,2021-04-05,2022-05-02,231.11,392.0
1,3.0,178.95,2021-04-20,2022-05-02,533.48,377.0
2,1.0,125.68,2021-04-05,2022-05-02,125.34,392.0
3,1.0,148.45,2021-04-05,2022-05-02,148.11,392.0
4,1.0,88.89,2021-04-03,2022-05-02,88.55,394.0


**REMARKS**

The data are already cleaned and prepared for use in this notebook. Thus, customary cleaning and preparation steps are skipped here.

---
## ANALYSES

### First, I generated frequency distributions to show the percentages of customers by **Historic Number Of Orders** and **Average Order Value**.

In [50]:
fig = make_subplots(rows=1, cols=2,
                   subplot_titles=("Historic Number Of Orders", "Average Order Value"))

fig.add_trace(
    go.Histogram(x=df['Historic Number Of Orders'], 
                 histnorm='percent', marker_color='#3F5E9B',
                opacity=0.7),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(x=df['Average Order Value'], histnorm='percent', 
                 nbinsx=20, 
                 marker_color='#758AB3',
                opacity=0.7),
    row=1, col=2
)

fig.update_layout(height=400, width=900, 
                  bargap=0.1, plot_bgcolor='white',
                  yaxis1_range=[0,100], 
                  yaxis2_range=[0,100],
                  showlegend=False
#                   title_text=""
                 )

fig.update_yaxes(title_text="Percent of Customers", row=1, col=1)
fig.update_yaxes(title_text="Percent of Customers", row=1, col=2)
fig.update_xaxes(title_text="No. of Orders", row=1, col=1)
fig.update_xaxes(title_text="Dollars", row=1, col=2)

fig.show()

**REMARKS**

* about 69% of customers have placed only one order
* only about 17% have placed 2 orders
* 58% of customers' average order value is between 100 and 199.90 dollars.

The low percentage of repeat business could be due to a high proportion of new customers -- i.e., customers who activated only recently. One might expect __Historic Number Of Orders__ to increase the longer someone is a customer. The strength of association between __days active__ and __Historic Number Of Orders__ is computed in an analysis below.

### Next, I generated a boxplot to visualize the scatter (spread) of Average Order Value, aggregated by Historic Number Of Orders per customer.

In [8]:
# Generate an interactive boxplot:
box_fig = px.box(data_frame = df,
       y = 'Average Order Value',
       x = 'Historic Number Of Orders',
       width=600, height=400,
      title='',
                 template="simple_white",
                color='Historic Number Of Orders'
                )

box_fig.update_xaxes(dtick=1)
box_fig.update_yaxes(tickprefix="$")

box_fig.layout.update(showlegend=False) 

box_fig.show()

# box_fig.write_html("/Users/cea/Desktop/avgval_x_orders.html")

**REMARKS**

Notably: the 4th quartile (*75th - 100th percentile*) __Average Order Value__ shrinks as __Historic Number Of Orders__ increases.

...... ......

For a different presentation of the data, I consolidated __Historic Number Of Orders__ and __Average Order Value__ data into a tabular format.

In [10]:
# Create a new dataframe that aggregates data by Historic Number Of Orders
df_order_val = df.groupby('Historic Number Of Orders')['Average Order Value'].agg(['median', 'min', 'max', 'count']).reset_index()

# Compute the percentages of customers who placed different number of orders
df_order_val['pct_of_total'] = df_order_val['count']/df_order_val['count'].sum()

# Apply string formatting to denote units
format_dict = {'Historic Number Of Orders': '{0:,.0f}', 
               'median':'${0:,.2f}', 'min': '${0:,.2f}', 
               'max': '${0:,.2f}', 'pct_of_total': '{:.2%}'}

df_order_val.style.format(format_dict).hide_index().bar(
    color='#33DDFF',
    vmin=0, subset=['pct_of_total'],
    align='zero').bar(
    color='#FFA07A',vmin=100, 
    subset=['median'],
    align='zero').background_gradient(
    subset=['max'], cmap='BuGn').set_caption(
    'Median, min and max of \'Average Order Value\', aggregated by Historic Number Of Orders')

Historic Number Of Orders,median,min,max,count,pct_of_total
1,$148.68,$1.51,"$1,073.61",1948,69.35%
2,$152.77,$54.29,$638.08,475,16.91%
3,$170.69,$1.51,$519.16,186,6.62%
4,$167.29,$78.40,$406.37,91,3.24%
5,$167.07,$98.78,$324.87,43,1.53%
6,$152.14,$99.41,$308.44,29,1.03%
7,$171.72,$135.87,$216.50,14,0.50%
8,$154.78,$130.74,$295.75,10,0.36%
9,$185.89,$142.95,$289.91,6,0.21%
10,$283.66,$283.66,$283.66,1,0.04%


**REMARKS**

The table incoporates visual formatting to convey key information and add appeal:

* Bar charts in the __median__ and __pct_of_total__ columns
* Shading in the __max__ column

With these enhancements, one can easily see that as Historic Number Of Orders increases from 1 to 6:
 * the median Average Order Value changes very little but
 * the max Average Order Value drops considerably

The table restates a finding already noted above: about 70% of customers placed only one order.

Is this due to a disproportianate number of new customers? Does __Historic Number Of Orders__ increase with length of time as a customer?

### To answer that question, I evaluated the linear relationship -- strength of association -- between __Historic Number Of Orders__ and __days active__.

In [12]:
# # Use scipy to compute Pearson's Correlation Coefficient(R):
active_histord_r = pearsonr(df['Historic Number Of Orders'], df['days active'])
# pearsonr returns a tuple containing Pearson’s R & 2-tailed p-value

# Generate a scatter plot
sca_fig = px.scatter(data_frame = df,
                     y = 'days active',
                     x = 'Historic Number Of Orders',
                     width=600, height=400,
                     template="simple_white",
                     trendline="ols",
                    trendline_color_override="#f1c40f")

sca_fig.update_xaxes(dtick=1, title_text='Historic Number Of Orders')
sca_fig.update_yaxes(title_text='days active')



sca_fig.update_layout(

    title={
        
        'text': f'Pearson\'s R: {active_histord_r[0]:.2f}',
        # Use index 0 to take only Pearson’s R value from the tuple
        
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

sca_fig.show()

**REMARKS**

__Historic Number Of Orders__ and __active period__ are only weakly correlated: __R = 0.27__.

**NOTE**: The maximum possible R value is 1, which signifies two variables are perfectly positively correlated.

Ideally, a strong positive correlation should exist between these variables.

*Targeted campaigns could be developed to reactivate non-repeat and lapsed customers.

*If successful, the campaigns would result in __a higher R value__.

*The rate of change in R could be used as an additional metric of performance. 
    
    * It would indicate how quickly customers are responding.

### Does __Historic Customer Lifetime Value__ increase with number of __days active__?

In [12]:
# Compute the Pearson's correlation coefficient:
active_histval_r = pearsonr(df['Historic Customer Lifetime Value'], df['days active'])

# Generate a scatter plot of the data:
sca_fig = px.scatter(data_frame = df,
                     y = 'Historic Customer Lifetime Value',
                     x = 'days active', 
                     width=600, height=400,
                     template="simple_white",
                     trendline="ols",
                    trendline_color_override="#f1c40f")

sca_fig.update_xaxes(dtick=50, title_text='days active')
sca_fig.update_yaxes(tickprefix="$", title_text='Historic Customer Lifetime Value')

sca_fig.update_layout(
    title={
        'text': f'Pearson\'s R: {active_histval_r[0]:.2f}',
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

sca_fig.show()

**REMARKS**

__Historic customer lifetime value__ and __number of days active__ are only weakly correlated: __R = 0.19__.

Ideally, a strong positive correlation should exist between these variables.

*Targeted campaigns could be developed to drive new or higher spending by longstanding customers (e.g., a loyalty or 'we miss you' program).

*If successful, the campaigns would result in a higher R value for these variables.

### Does Historic Customer Lifetime Value increase with Historic Number Of Orders?

In [13]:
# Compute the Pearson's correlation coefficient:
histord_histval_r = pearsonr(df['Historic Number Of Orders'], df['Historic Customer Lifetime Value'])

sca_fig = px.scatter(data_frame = df,
                     y = 'Historic Customer Lifetime Value',
                     x = 'Historic Number Of Orders',
                    title=f'Pearson\'s R: {histord_histval_r[0]:.2f}',
                     width=600, height=400,
                     template="simple_white",
                     trendline="ols",
                    trendline_color_override="#f1c40f")

sca_fig.update_xaxes(dtick=1, title_text='Historic Number Of Orders')
sca_fig.update_yaxes(tickprefix="$", title_text='Historic Customer Lifetime Value')

sca_fig.update_layout(
    title={
        'text': f'Pearson\'s R: {histord_histval_r[0]:.2f}',
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

sca_fig.show()

__Historic customer lifetime value__ and __Historic number of orders__ are strongly correlated: **R = 0.86**.

Current business strategies and methods are performing well in support of this metric. 

As previously noted, however, a supermajority of customers (69%) placed only 1 order. 

*Implementation of strategies to drive repeat business could help improve the __R__ value for these variables.